In [8]:
import pandas as pd
import numpy as np

raw_data = pd.read_csv("bank.csv", sep=';')
raw_data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [9]:
from sklearn.preprocessing import LabelEncoder # Chuyển thuộc tính dạng category thành số.
def categoryToInt(data: pd.DataFrame):
    listAttrCate = ['job', 'marital', 'education', 'default', 'housing', 'month', 'loan', 'contact', 'poutcome', 'y']
    new_data = data.copy()
    LE = LabelEncoder()
    for attr in listAttrCate:
        new_data[attr] = LE.fit_transform(new_data[attr])
    return new_data

new_data = categoryToInt(raw_data)

In [10]:
from scipy import stats

z = np.abs(stats.zscore(new_data))
revmove_outlier_data = new_data[(z < 3).all(axis=1)]
revmove_outlier_data.shape

(3908, 17)

In [11]:
X = revmove_outlier_data.values[:,:-1]
y = revmove_outlier_data.values[:,-1:].flatten()# flatten copy còn ravel không copy
listAttrName = revmove_outlier_data.columns.tolist() # lấy danh sách tên thuộc tính (vẽ trong decision tree)
print('X[0]: ', *X[0])
print('len X[0]: ', len(X[0]))
print('Class 0: ', len(y[y==0]))
print('Class 1: ', len(y[y==1]))

X[0]:  30 10 1 0 0 1787 0 0 0 19 10 79 1 -1 0 3
len X[0]:  16
Class 0:  3515
Class 1:  393


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
# lấy 10% là test
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print('X_train[0] : ', *X_train[0])

(3517, 16)
(391, 16)
(3517,)
(391,)
X_train[0] :  27 6 1 1 0 -220 1 0 2 26 8 138 2 -1 0 3


In [13]:
## https://stackoverflow.com/questions/51237635/difference-between-standard-scaler-and-minmaxscaler#:~:text=StandardScaler%20removes%20the%20mean%20and%20scales%20the%20data%20to%20unit%20variance.&text=MinMaxScaler%20rescales%20the%20data%20set,the%20transformed%20number%20of%20households.
## https://stackoverflow.com/questions/49444262/normalize-data-before-or-after-split-of-training-and-testing-data
# Scale sau bước train_test_split để không rò rỉ dữ liệu?

#Scale MinMax
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
print('X_train[0] : ', *X_train[0], sep='\t') # Có thể so sánh với đối tượng X_train thứ 0 ở trên trước khi scale

X_train[0] : 	0.15094339622641512	0.5454545454545454	0.5	0.3333333333333333	0.0	0.1258660508083141	1.0	0.0	1.0	0.8333333333333334	0.7272727272727273	0.13009708737864076	0.09090909090909091	0.0	0.0	1.0


In [14]:
from sklearn.neural_network import MLPClassifier
model_identity = MLPClassifier(activation='identity', hidden_layer_sizes=(5, 5, 2), max_iter=1000, random_state=42)
model_identity.fit(X_train, y_train)

model_logistic = MLPClassifier(activation='logistic', hidden_layer_sizes=(5, 5, 2), max_iter=1000, random_state=42)
model_logistic.fit(X_train, y_train)

model_tanh = MLPClassifier(activation='tanh', hidden_layer_sizes=(5, 5, 2), max_iter=1000, random_state=42)
model_tanh.fit(X_train, y_train)

model_relu = MLPClassifier(hidden_layer_sizes=(5, 5, 2), max_iter=1000, random_state=42)
model_relu.fit(X_train, y_train)

# weight optimization = adam’ refers to a stochastic gradient-based optimizer proposed by Kingma, Diederik, and Jimmy Ba

y_pred_iden = model_identity.predict(X_test)
y_pred_logi = model_logistic.predict(X_test)
y_pred_tanh = model_tanh.predict(X_test)
y_pred_relu = model_relu.predict(X_test)


In [15]:
# Scale minmax
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score

print('----------identity')
print('accuracy:\n', accuracy_score(y_test, y_pred_iden)*100)
print(confusion_matrix(y_test, y_pred_iden))
print(classification_report(y_test, y_pred_iden))

print('----------logistic')
print('accuracy:\n', accuracy_score(y_test, y_pred_logi)*100)
print(confusion_matrix(y_test, y_pred_logi))
print(classification_report(y_test, y_pred_logi))

print('----------tanh')
print('accuracy:\n', accuracy_score(y_test, y_pred_tanh)*100)
print(confusion_matrix(y_test, y_pred_tanh))
print(classification_report(y_test, y_pred_tanh))

print('----------relu')
print('accuracy:\n', accuracy_score(y_test, y_pred_relu)*100)
print(confusion_matrix(y_test, y_pred_relu))
print(classification_report(y_test, y_pred_relu))

----------identity
accuracy:
 10.485933503836318
[[  4 350]
 [  0  37]]
              precision    recall  f1-score   support

           0       1.00      0.01      0.02       354
           1       0.10      1.00      0.17        37

    accuracy                           0.10       391
   macro avg       0.55      0.51      0.10       391
weighted avg       0.91      0.10      0.04       391

----------logistic
accuracy:
 90.53708439897699
[[354   0]
 [ 37   0]]
              precision    recall  f1-score   support

           0       0.91      1.00      0.95       354
           1       0.00      0.00      0.00        37

    accuracy                           0.91       391
   macro avg       0.45      0.50      0.48       391
weighted avg       0.82      0.91      0.86       391

----------tanh
accuracy:
 14.578005115089516
[[ 20 334]
 [  0  37]]
              precision    recall  f1-score   support

           0       1.00      0.06      0.11       354
           1       0.10   

C:\Users\lamph\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
# Non scaling
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score

print('----------identity')
print('accuracy:\n', accuracy_score(y_test, y_pred_iden)*100)
print(confusion_matrix(y_test, y_pred_iden))
print(classification_report(y_test, y_pred_iden))

print('----------logistic')
print('accuracy:\n', accuracy_score(y_test, y_pred_logi)*100)
print(confusion_matrix(y_test, y_pred_logi))
print(classification_report(y_test, y_pred_logi))

print('----------tanh')
print('accuracy:\n', accuracy_score(y_test, y_pred_tanh)*100)
print(confusion_matrix(y_test, y_pred_tanh))
print(classification_report(y_test, y_pred_tanh))

print('----------relu')
print('accuracy:\n', accuracy_score(y_test, y_pred_relu)*100)
print(confusion_matrix(y_test, y_pred_relu))
print(classification_report(y_test, y_pred_relu))

----------identity
accuracy:
 86.18925831202046
[[321  33]
 [ 21  16]]
              precision    recall  f1-score   support

           0       0.94      0.91      0.92       354
           1       0.33      0.43      0.37        37

    accuracy                           0.86       391
   macro avg       0.63      0.67      0.65       391
weighted avg       0.88      0.86      0.87       391

----------logistic
accuracy:
 90.53708439897699
[[354   0]
 [ 37   0]]
              precision    recall  f1-score   support

           0       0.91      1.00      0.95       354
           1       0.00      0.00      0.00        37

    accuracy                           0.91       391
   macro avg       0.45      0.50      0.48       391
weighted avg       0.82      0.91      0.86       391

----------tanh
accuracy:
 90.53708439897699
[[354   0]
 [ 37   0]]
              precision    recall  f1-score   support

           0       0.91      1.00      0.95       354
           1       0.00     

C:\Users\lamph\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
